# Assignment 7 - Network Flow
## Edmonds-Karp

Below is the array representing the flow network shown in the assignment PDF. Along with it is a list of node labels the node labels.

In [2]:
capacities = [
    #a, b, c, d, e, f
    [0, 0, 9, 9, 0, 0],  # a
    [9, 0, 0, 4, 5, 0],  # b
    [0, 9, 0, 0, 0, 0],  # c
    [0, 0, 0, 0, 9, 4],  # d
    [0, 0, 3, 0, 0, 9],  # e
    [0, 0, 5, 0, 0, 0],  # f
]
node_labels = ["a", "b", "c", "d", "e", "f"]

### Helper Functions
First we are going to create a couple helper functions to make implementing the algorithm a little easier, these are already implemented for you. If you decide on storing the path a different way than what is assumed in the function `path_to_str()` you need to update this function such that the output will be the same (this will make it easier for the TAs to verify that your solution is correct).

In [3]:
def to_str(node_index: int) -> str:
    return node_labels[node_index]


def to_index(node_label: str) -> int:
    return node_labels.index(node_label)


def path_to_str(path: list) -> str:
    """convert a `path (list[int])` to a easy to read string of the visited nodes
    for instance: "abcdef"""
    if path is None:
        return "None"
    return "".join([to_str(node_index) for node_index in path])


# Testing the function
my_path = [0, 1, 2, 3, 4, 5]
print(f"{my_path} => {path_to_str(my_path)}")


[0, 1, 2, 3, 4, 5] => abcdef


Next we create another helper function. This one is python specific to prevent us rewriting our original capacities array. If we don't create a duplicate/separate array the algorithm alters, you will have to rerun the cell that initializes the original array every time you want to rerun your algorithm, which is tedious. Using this function is already included in the `edmonds_karp()` skeleton, just know why we do it. In different languages or circumstances creating the duplicate array might not be necessary.

In [4]:
def duplicate(lst: list) -> list:
    """Duplicate/make copy of the list given,
    should be used to not edit original`capacities` during the algorithm
    se we can rerun each cell individually without resetting entire notebook
    
    Args:
        `lst (list[list[int]])`: the list to duplicate"""
    return [row[:] for row in lst]


# Testing the function
print(f"{capacities}\n{duplicate(capacities)}")


[[0, 0, 9, 9, 0, 0], [9, 0, 0, 4, 5, 0], [0, 9, 0, 0, 0, 0], [0, 0, 0, 0, 9, 4], [0, 0, 3, 0, 0, 9], [0, 0, 5, 0, 0, 0]]
[[0, 0, 9, 9, 0, 0], [9, 0, 0, 4, 5, 0], [0, 9, 0, 0, 0, 0], [0, 0, 0, 0, 9, 4], [0, 0, 3, 0, 0, 9], [0, 0, 5, 0, 0, 0]]


### Implementing Edmonds-Karp
With that out of the way, it's time to implement our Edmonds-Karp algorithm. Edmonds-Karp, put simply, is just Ford-Fulkerson but instead of choosing augmenting paths "randomly", the shortest path from the source to the sink is always used, making the output of Edmond-Karp deterministic. To do this we will use our old friend BFS. 

> Tip: You can reuse your BFS from assignment 2, it should just need some minor tweaks to adapt it to the flow network.

Implement the BFS and verify that it finds the shortest path in the given flow network. Note that the function uses the indices of the start/end node in the capacities (`C`) array.

In [24]:
def BFS(C: list, start: int, end: int) -> str:
    """
    Args:
        `C (list[list[int]])`: Capacities
        `start (int)`: start index
        `end (int)`: end index
    """

    shortest_paths = {
        to_str(start) : [start]
    }
    # nodes already checked
    visited = []
    # nodes "discovered" through the graph, yet to be checked
    queue = [start]

    while queue:
        current_node = queue.pop(0)

        if current_node == end:
            return shortest_paths[to_str(current_node)]

        visited.append(current_node)

        for next_node in range(len(C[current_node])):
            if C[current_node][next_node] > 0 and next_node not in visited:
                queue.append(next_node)

                path = shortest_paths[to_str(current_node)].copy()
                path.append(next_node)

                if to_str(next_node) not in shortest_paths:
                    shortest_paths[to_str(next_node)] = path
                
    return None


# Testing the function (from "a" to "f")
path = path_to_str(BFS(capacities, start=to_index("a"), end=to_index("f")))
print(path)


adf


Now that we can find the shortest path, we need to actually increase the flow through this path and update the capacities accordingly.

Implement this in the function below. The function should find the correct flow for the given path, update the correct values, and return the flow value.

In [23]:
def update_capacities(C: list, path: list) -> int:
    """
    Args:
        `C (list[list[int]])`: Capacities
        `path (list[int])`: path to update capacities along
    """

    bottleneck_value = float("inf")

    for i in range(len(path)-1):
    
        current_index = to_index(path[i])
        next_index = to_index(path[i+1])

        if C[current_index][next_index] > 0 and (C[current_index][next_index] < bottleneck_value):
            bottleneck_value = C[current_index][next_index]


    return bottleneck_value

update_capacities(capacities, path)

4

We are now all set to implement our Edmonds-Karp algorithm. Implement the function (using the functions created above) to find the maximum flow through the flow network given a source index and a sink index.

In [41]:
def edmonds_karp(C: list, source: int, sink: int):
    """
    Args:
        `C (list[list[int]])`: Capacities
        `source (int)`: start index/s
        `sink (int)`: end index/t
    """
    C = duplicate(C)  # duplicate to not alter original array
    max_flow = 0

    while True:
        path =  path_to_str(BFS(C, source, sink))
        

        if path == "None":
            break

        bottleneck_value = update_capacities(C, path)

        for i in range(len(path)-1):
            current_index = to_index(path[i])
            next_index = to_index(path[i+1])

            C[current_index][next_index] -= bottleneck_value
            C[next_index][current_index] += bottleneck_value

        max_flow += bottleneck_value

    return max_flow


max_flow = edmonds_karp(capacities, source=to_index("b"), sink=to_index("f"))
print(f"Maximum Flow (b-f): {max_flow}")

Maximum Flow (b-f): 13


In [42]:
# Finding maximum flow between all nodes (used to verify your solution)
for i in range(len(capacities)):
    for j in range(len(capacities[i])):
        if i == j:
            continue
        max_flow = edmonds_karp(capacities, i, j)
        print(f"Maximum Flow ({to_str(i)}-{to_str(j)}) is {max_flow}")

Maximum Flow (a-b) is 9
Maximum Flow (a-c) is 17
Maximum Flow (a-d) is 13
Maximum Flow (a-e) is 14
Maximum Flow (a-f) is 13
Maximum Flow (b-a) is 9
Maximum Flow (b-c) is 17
Maximum Flow (b-d) is 13
Maximum Flow (b-e) is 14
Maximum Flow (b-f) is 13
Maximum Flow (c-a) is 9
Maximum Flow (c-b) is 9
Maximum Flow (c-d) is 9
Maximum Flow (c-e) is 9
Maximum Flow (c-f) is 9
Maximum Flow (d-a) is 8
Maximum Flow (d-b) is 8
Maximum Flow (d-c) is 8
Maximum Flow (d-e) is 13
Maximum Flow (d-f) is 13
Maximum Flow (e-a) is 8
Maximum Flow (e-b) is 8
Maximum Flow (e-c) is 8
Maximum Flow (e-d) is 8
Maximum Flow (e-f) is 12
Maximum Flow (f-a) is 5
Maximum Flow (f-b) is 5
Maximum Flow (f-c) is 5
Maximum Flow (f-d) is 5
Maximum Flow (f-e) is 5
